In [1]:
# install required packages. 
# xgboost took long time to compile (10min?) in linux. Recommend using terminal to install to see progress
install.packages("xgboost")


Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)


In [2]:
library(tidyverse)
options(digits=3)
# load all implemented methods
source("methods/BaselineUnAdjusted/baselineUnajusted.R")
source("methods/knownprior/knownprior.R")
source("methods/split/simple_Splitting.R")
source("methods/postselect/postselect.R")
source("methods/split/boost_Splitting.R")
source("methods/EBmle/ebmle.R")
source("methods/BFBound/bfbound.R")
source("methods/MixturePrior/mixture.R")
source("methods/split/TARwES.R")
source("methods/EBHuberSURE/EBHuberSURE.R")
source("methods/postselect/postselect.R")

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.2.0     ✔ purrr   0.3.3
✔ tibble  2.1.3     ✔ dplyr   0.8.1
✔ tidyr   0.8.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘xgboost’

The following object is masked from ‘package:dplyr’:

    slice



In [3]:
# Simulation Procedure. please check this file for details about simulation setup and simulation main function *evaluateSim*
source("simulation/simulation.R")

In [12]:
# Define list of method to test
# baselineAnadjust: unadjusted naive MLE
# simpleSplitting: RwES with linear regression
# TARwESSplitting: TARwES
# NormalPriorEB: EB using normal prior (James-Stein)
# LaplacePriorEB: EB using Laplace prior
# ZeroNormalLaplaceMixEB: Ghidorah with three heads
# HuberPriorEB: EB using Huber prior
# GhidorahSplitting: TARwES+ where Ghidorah is used as one of the feature
# postSelectZCut(1.96): CMLE using z-score 1.96 as cutoff

# With CMLE the simulation study could run quite slow. I recommend to run without CMLE at first if you are impatient :)
methodlist = c(baselineUnadjust, simpleSplitting, bstSplitting, TARwESSplitting, NormalPriorEB, LaplacePriorEB, ZeroNormalLaplaceMixEB, HuberPriorEB, GhidorahSplitting, postSelectZCut(1.96))
#methodlist = c(baselineUnadjust, simpleSplitting,bstSplitting, TARwESSplitting, NormalPriorEB, LaplacePriorEB, ZeroNormalLaplaceMixEB, HuberPriorEB, GhidorahSplitting)

In [13]:
# Simulation Study 1
# note the set.seed for replication
set.seed(1)
res = evaluateSim(200, 100, 1000, partial(rnorm,mean=0, sd=0.1), partial(sample,x=c(20,50,100,200),replace=TRUE), scale = 1, methods = c(KnownNormalPrior(0.1),methodlist))
res
                    

Selection,KnownNormalPrior,baselineUnadjust,RwES-LinearReg,bstSplitting,TARwES,NormalPriorEB(J-S),LaplacePriorEB,Ghidorah,HuberPriorEB,TARwES+,postSelect(|z|>1.96),selectionRate
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
p<0.01,0.0671,0.217,0.1066,0.1219,0.0826,0.0751,0.0778,0.0761,0.0704,0.0859,0.170,1.0000
p<0.05,0.0706,0.218,0.0998,0.1066,0.0807,0.0768,0.0777,0.0765,0.0731,0.0831,0.131,0.1510
All,0.0763,0.146,0.0857,0.0907,0.0787,0.0781,0.0782,0.0781,0.0770,0.0795,0.106,0.0648
p<0.01-CI,0.9518,0.709,0.8750,0.8148,0.9557,0.9105,0.9698,0.9523,0.9289,0.9485,1.000,1.0000
p<0.05-CI,0.9512,0.781,0.9240,0.8953,0.9731,0.9167,0.9633,0.9503,0.9308,0.9690,1.000,0.1510
All-CI,0.9507,0.950,0.9799,0.9711,0.9913,0.9317,0.9229,0.9153,0.9374,0.9904,1.000,0.0648
p<0.01-VR,0.5425,1.000,1.0000,1.0000,1.0000,0.5378,0.8414,0.7131,0.5274,1.0000,5.926,1.0000
p<0.05-VR,0.4991,1.000,1.0000,1.0000,1.0000,0.4964,0.7105,0.6381,0.4850,1.0000,5.522,0.1510
All-VR,0.4163,1.000,1.0000,1.0000,1.0000,0.4174,0.4037,0.3862,0.4043,1.0000,4.167,0.0648


In [14]:
resx = evaluateSim(200, 1000, 1000, partial(rnorm,mean=0, sd=0.1), partial(sample,x=c(20,50,100,200),replace=TRUE), scale = 1, methods = c(KnownNormalPrior(0.1),methodlist))
resx

Selection,KnownNormalPrior,baselineUnadjust,RwES-LinearReg,bstSplitting,TARwES,NormalPriorEB(J-S),LaplacePriorEB,Ghidorah,HuberPriorEB,TARwES+,postSelect(|z|>1.96),selectionRate
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
p<0.01,0.0686,0.219,0.1026,0.0956,0.0706,0.0695,0.0737,0.0705,0.0689,0.0711,0.172,1.0000
p<0.05,0.0711,0.217,0.0948,0.0879,0.0724,0.0718,0.0738,0.0722,0.0714,0.0728,0.132,0.1521
All,0.0765,0.146,0.0831,0.0826,0.0768,0.0767,0.0773,0.0769,0.0766,0.0769,0.107,0.0651
p<0.01-CI,0.9455,0.709,0.8908,0.9137,0.9859,0.9421,0.9777,0.9620,0.9434,0.9852,0.999,1.0000
p<0.05-CI,0.9471,0.777,0.9346,0.9495,0.9896,0.9443,0.9762,0.9626,0.9455,0.9891,1.000,0.1521
All-CI,0.9491,0.949,0.9829,0.9845,0.9939,0.9464,0.9338,0.9414,0.9474,0.9938,1.000,0.0651
p<0.01-VR,0.5410,1.000,1.0000,1.0000,1.0000,0.5359,0.8516,0.6563,0.5372,1.0000,5.927,1.0000
p<0.05-VR,0.5003,1.000,1.0000,1.0000,1.0000,0.4954,0.7195,0.5993,0.4966,1.0000,5.528,0.1521
All-VR,0.4166,1.000,1.0000,1.0000,1.0000,0.4124,0.4021,0.4076,0.4133,1.0000,4.171,0.0651


In [15]:
# Simulation study 2
# please run this right after the last cell to replicate result (no set.seed here)

res2 =  evaluateSim(200, 100, 1000,  partial(r_zeroInflated, p = 0.5, generator = partial(rt_scale,df=3, ncp=0, sd=0.2)), partial(sample,x=c(20,50,100,200),replace=TRUE), scale = 1, methods = c(KnownZeroInflatedTPrior(3, 0.2/sqrt(3),0.5),methodlist))
res2


Selection,KnownZeroInflatedTPrior,baselineUnadjust,RwES-LinearReg,bstSplitting,TARwES,NormalPriorEB(J-S),LaplacePriorEB,Ghidorah,HuberPriorEB,TARwES+,postSelect(|z|>1.96),selectionRate
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
p<0.01,0.1438,0.199,0.239,0.358,0.1594,0.201,0.1355,0.1321,0.1918,0.1648,0.165,1.0000
p<0.05,0.1430,0.220,0.198,0.276,0.1432,0.170,0.1286,0.1252,0.1633,0.1462,0.145,0.1446
All,0.0893,0.146,0.109,0.133,0.0882,0.098,0.0844,0.0819,0.0952,0.0882,0.108,0.0754
p<0.01-CI,0.8855,0.799,0.691,0.526,0.8571,0.715,0.8924,0.9063,0.7120,0.8438,1.000,1.0000
p<0.05-CI,0.8840,0.718,0.814,0.708,0.9084,0.790,0.9130,0.9207,0.7892,0.8999,1.000,0.1446
All-CI,0.9686,0.950,0.966,0.943,0.9809,0.940,0.9506,0.9154,0.9389,0.9784,1.000,0.0754
p<0.01-VR,1.0000,1.000,1.000,1.000,1.0000,0.614,0.9063,0.9414,0.6107,1.0000,5.470,1.0000
p<0.05-VR,1.0000,1.000,1.000,1.000,1.0000,0.581,0.7999,0.8666,0.5741,1.0000,5.354,0.1446
All-VR,1.0000,1.000,1.000,1.000,1.0000,0.523,0.4847,0.3699,0.5106,1.0000,4.114,0.0754


In [16]:
res2x =  evaluateSim(200, 1000, 1000,  partial(r_zeroInflated, p = 0.5, generator = partial(rt_scale,df=3, ncp=0, sd=0.2)), partial(sample,x=c(20,50,100,200),replace=TRUE), scale = 1, methods = c(KnownZeroInflatedTPrior(3, 0.2/sqrt(3),0.5),methodlist))
res2x

Selection,KnownZeroInflatedTPrior,baselineUnadjust,RwES-LinearReg,bstSplitting,TARwES,NormalPriorEB(J-S),LaplacePriorEB,Ghidorah,HuberPriorEB,TARwES+,postSelect(|z|>1.96),selectionRate
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
p<0.01,0.1471,0.198,0.215,0.266,0.1355,0.1847,0.1327,0.1263,0.1765,0.1294,0.165,1.0000
p<0.05,0.1442,0.218,0.180,0.219,0.1270,0.1590,0.1259,0.1207,0.1526,0.1225,0.145,0.1459
All,0.0893,0.146,0.102,0.111,0.0831,0.0944,0.0832,0.0801,0.0917,0.0806,0.108,0.0765
p<0.01-CI,0.8814,0.795,0.723,0.683,0.9131,0.7539,0.9067,0.9274,0.7577,0.9209,1.000,1.0000
p<0.05-CI,0.8829,0.714,0.837,0.798,0.9425,0.8199,0.9290,0.9382,0.8235,0.9428,1.000,0.1459
All-CI,0.9689,0.950,0.971,0.961,0.9867,0.9518,0.9603,0.9253,0.9513,0.9848,1.000,0.0765
p<0.01-VR,1.0000,1.000,1.000,1.000,1.0000,0.6432,0.9253,0.9830,0.6472,1.0000,5.473,1.0000
p<0.05-VR,1.0000,1.000,1.000,1.000,1.0000,0.6092,0.8252,0.9104,0.6096,1.0000,5.350,0.1459
All-VR,1.0000,1.000,1.000,1.000,1.0000,0.5472,0.5040,0.3797,0.5351,1.0000,4.115,0.0765


In [17]:
# Simulation study 3
# please run this right after the last cell to replicate result (no set.seed here)

res3 =  evaluateSim(200, 100, 1000,  partial(r_zeroInflated, p = 0.9, generator = partial(rt_scale,df=3, ncp=0, sd=1)), partial(sample,x=c(20,50,100,200),replace=TRUE), scale = 1, methods = c(KnownZeroInflatedTPrior(3, 1/sqrt(3),0.9),methodlist))
res3


Selection,KnownZeroInflatedTPrior,baselineUnadjust,RwES-LinearReg,bstSplitting,TARwES,NormalPriorEB(J-S),LaplacePriorEB,Ghidorah,HuberPriorEB,TARwES+,postSelect(|z|>1.96),selectionRate
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
p<0.01,0.1644,0.199,0.391,1.044,0.246,0.348,0.1947,0.1662,0.339,0.2059,0.176,1.0000
p<0.05,0.1563,0.241,0.345,0.828,0.229,0.306,0.1950,0.1574,0.301,0.1861,0.165,0.1145
All,0.0651,0.145,0.151,0.294,0.103,0.134,0.0955,0.0657,0.132,0.0753,0.105,0.0706
p<0.01-CI,0.8991,0.822,0.535,0.220,0.723,0.568,0.8099,0.9007,0.566,0.8142,0.999,1.0000
p<0.05-CI,0.9057,0.578,0.651,0.454,0.799,0.592,0.8370,0.9063,0.594,0.8621,1.000,0.1145
All-CI,0.9845,0.950,0.957,0.912,0.975,0.949,0.9763,0.9703,0.949,0.9805,1.000,0.0706
p<0.01-VR,1.0000,1.000,1.000,1.000,1.000,0.765,0.9757,0.9973,0.767,1.0000,4.750,1.0000
p<0.05-VR,1.0000,1.000,1.000,1.000,1.000,0.753,0.9204,0.8969,0.751,1.0000,4.915,0.1145
All-VR,1.0000,1.000,1.000,1.000,1.000,0.739,0.6484,0.2053,0.727,1.0000,3.996,0.0706


In [18]:
res3x =  evaluateSim(200, 1000, 1000,  partial(r_zeroInflated, p = 0.9, generator = partial(rt_scale,df=3, ncp=0, sd=1)), partial(sample,x=c(20,50,100,200),replace=TRUE), scale = 1, methods = c(KnownZeroInflatedTPrior(3, 1/sqrt(3),0.9),methodlist))
res3x

Selection,KnownZeroInflatedTPrior,baselineUnadjust,RwES-LinearReg,bstSplitting,TARwES,NormalPriorEB(J-S),LaplacePriorEB,Ghidorah,HuberPriorEB,TARwES+,postSelect(|z|>1.96),selectionRate
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
p<0.01,0.1612,0.192,0.289,0.755,0.1947,0.305,0.1779,0.1593,0.281,0.1616,0.170,1.0000
p<0.05,0.1540,0.238,0.273,0.611,0.1955,0.279,0.1878,0.1521,0.261,0.1536,0.161,0.1140
All,0.0647,0.145,0.134,0.211,0.0968,0.130,0.0969,0.0642,0.126,0.0647,0.105,0.0705
p<0.01-CI,0.9033,0.832,0.601,0.440,0.7973,0.648,0.8375,0.9115,0.668,0.9053,0.999,1.0000
p<0.05-CI,0.9100,0.584,0.708,0.607,0.8506,0.619,0.8528,0.9143,0.632,0.9169,1.000,0.1140
All-CI,0.9850,0.951,0.965,0.951,0.9813,0.954,0.9793,0.9702,0.955,0.9862,1.000,0.0705
p<0.01-VR,1.0000,1.000,1.000,1.000,1.0000,0.825,0.9874,0.9993,0.837,1.0000,4.754,1.0000
p<0.05-VR,1.0000,1.000,1.000,1.000,1.0000,0.813,0.9463,0.9020,0.825,1.0000,4.918,0.1140
All-VR,1.0000,1.000,1.000,1.000,1.0000,0.801,0.6966,0.1872,0.806,1.0000,3.994,0.0705
